In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# For LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [11]:
# created langchain environment in conda and selected that as kernel to run the code
from langchain_openai import ChatOpenAI

# no need to pass api key as we already loaded it from .env file
llm=ChatOpenAI(model='gpt-3.5-turbo')
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000223097C8A90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000223097E05B0> root_client=<openai.OpenAI object at 0x000002230971C5E0> root_async_client=<openai.AsyncOpenAI object at 0x00000223097C8CD0> model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
# passing input and getting response from the llm
result=llm.invoke("explain gen ai")
print(result)

#### PromptTemplate
PromptTemplate is something using which we can instruct our llm model how to behave. PromptTemplate take as input a dictionary, where each key represents a variable in the prompt template to fill in. 
- ChatPromptTemplate

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question"),
    ("user","{input}")
])
print(prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


#### Reterieval and document chain: Langchain Expression Language (LCEL)
- Chaining using | operator: this | is saying that we need to combine all things connected with | and use it in passing to llm
- create_stuff_doccuments_chain(pass whatever you what to combine): it is a runnable binding

In [ ]:
chain=prompt|llm
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

#### Output Parser
It is responsible for providing the response how we want to display it. 
We can also create our own custom output parsers.
- StrOutputParser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain=prompt|llm|output_parser
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

#### Simple GEN AI App using OpenAI
We are going to scrape one web page and then ask answer based on that using llm

Steps:
- load .env file api keys
- load or scrap the data from web page - DataLoaders
- create chunk of loaded document - Splitting
- using embedding convert the chunk in vector stores
- querying the vector store using similarity search
- create llm , prompt and use retrieval chain - important 
- reterivers using create_retrieval_chain()
- get the response by invoking the retriever_chain created


In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
web_content=loader.load()
print(web_content)

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a Simple LLM Application | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM ApplicationBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Ex

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

web_split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
web_chunks=web_split.split_documents(web_content)
print(web_chunks)

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a Simple LLM Application | 🦜️🔗 LangChain'), Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM applicati

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

openai_embeddings=OpenAIEmbeddings()
faiss_db=FAISS.from_documents(web_chunks,openai_embeddings)

In [ ]:
query="question related to loaded documents"
result=faiss_db.similarity_search(query)
print(result[0].page_content)

In [ ]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI()

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the question based on provided context.
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm, prompt)
print(document_chain)

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"your question here",
    "context": [Document(page_content="context you want to provide")]
})

#### if using reterievers then after creating vectorstores convert that in retrievers

In [ ]:
retriever=faiss_db.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain

# using this, the context automatically gets in prompt template
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
final_response=retriever_chain.invoke({"input":"question here"})
print(final_response)

### Simple Gen AI App using Ollama

In [29]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

ollama_embedding=OllamaEmbeddings(model='llama3')
ollama_faiss_db=FAISS.from_documents(web_chunks,ollama_embedding)

C:\Users\srish\AppData\Local\Temp\ipykernel_27456\2621887515.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embedding=OllamaEmbeddings(model='llama3')


In [36]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

ollama_llm=Ollama(model="llama3")
ollama_prompt=ChatPromptTemplate.from_template(
    """ 
Answer the question based on the provided context.
<context>
{context}
</context>
"""
)

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

ollama_doc_chain=create_stuff_documents_chain(ollama_llm,ollama_prompt)

In [38]:
ollama_retriever=ollama_faiss_db.as_retriever()

In [41]:
from langchain.chains import create_retrieval_chain

ollama_retriever_chain=create_retrieval_chain(ollama_retriever,ollama_doc_chain)
ollama_response=ollama_retriever_chain.invoke({"input":"jupyter setup"})
print(ollama_response)
print(ollama_response['answer'])

{'input': 'jupyter setup', 'context': [Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='ConversationBufferWindowMemory or ConversationTokenBufferMemoryMigrating off ConversationSummaryMemory or ConversationSummaryBufferMemoryA Long-Term Memory AgentRelease policySecurityTutorialsBuild a Simple LLM ApplicationOn this pageBuild a Simple LLM Application'), Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Sim